In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
!pip install folium
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

distributed 1.21.8 requires msgpack, which is not installed.
Libraries imported.


In [24]:
# To extract from Wikipedia page
from bs4 import BeautifulSoup
url=  requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
#with open (url) as html_file:
data = BeautifulSoup(url.text,'lxml')
#print(data.prettify())


In [25]:
#parse data and save to csv file
import csv
csv_file=open('canada.csv','w')
csv_writer=csv.writer(csv_file)
csv_writer.writerow(['Postcode', 'Borough', 'Neighbourhood'])
for tr in data.find_all('tr')[1:]:
    tds = tr.find_all('td')
    if len(tds)==3:
        Postcode=tds[0].text
        Borough=tds[1].text
        Neighbourhood=tds[2].text
    #print(Postcode, Borough, Neighbourhood)
    csv_writer.writerow([Postcode, Borough, Neighbourhood])
csv_file.close()

In [26]:
#load canada data from CSV
import pandas as pd
df_Canada=pd.read_csv('canada.csv')
print('Data loaded')

Data loaded


In [27]:
#removes \n at the end
df_Canada[df_Canada.columns] = df_Canada.apply(lambda x: x.str.strip('\n'))
df_Canada.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [28]:
# exclude not assigned borough 
df_Canada_Assigned=df_Canada[df_Canada.Borough!='Not assigned']
df_Canada_Assigned.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [29]:
#Group neighbourhoods by postal code
df_Canada_Assigned.set_index(['Postcode','Borough'],inplace=True)
result = df_Canada_Assigned.groupby(level=['Postcode','Borough'], sort=False).agg( ','.join)
result.reset_index() # to remove set index

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Not assigned
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge,Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens,Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson,Garden District"


In [30]:
#remove not assignet neighbourhood
result.loc[result['Neighbourhood'] =='Not assigned']=result[result['Neighbourhood'] == 'Not assigned'].index.values[0][1]
result.head()

,,Neighbourhood
Postcode,Borough,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,"Harbourfront,Regent Park"
M6A,North York,"Lawrence Heights,Lawrence Manor"
M7A,Queen's Park,Queen's Park


In [31]:
#make a dataframe for canada
df_Canada_final=result.reset_index()
df_Canada_final.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [33]:
df_Canada_final.shape 

(103, 3)

In [35]:
#format address to be searchable in google maps
df_Canada_final["Address"]=df_Canada_final['Borough'] +", " +df_Canada_final['Postcode'] 
df_Canada_final.head()

,Postcode,Borough,Neighbourhood,Address
0,M3A,North York,Parkwoods,"North York, M3A"
1,M4A,North York,Victoria Village,"North York, M4A"
2,M5A,Downtown Toronto,"Harbourfront,Regent Park","Downtown Toronto, M5A"
3,M6A,North York,"Lawrence Heights,Lawrence Manor","North York, M6A"
4,M7A,Queen's Park,Queen's Park,"Queen's Park, M7A"


In [55]:
#url for google search
url2 = [] #generate list of urls
for i in range(0, len(df_Canada_final)):
    url = 'https://maps.googleapis.com/maps/api/geocode/json?address={},+Toronto+Ontario,+CA&key=YOUR_GOOGLE_API_KEY'.format(df_Canada_final['Address'][i])
    url2.append(url)
#url2

In [56]:
#complete df with latitude and longitude
df_Canada_final['Latitude']=0
df_Canada_final['Longitude']=0
for i, url in enumerate(url2):
    r = requests.get(url2[i])
    results = r.json()['results']
    #print (r.json())
    if results:
        location = results[0]['geometry']['location']
        df_Canada_final['Latitude'].iloc[i] = location['lat']
        df_Canada_final['Longitude'].iloc[i]= location['lng']
    else:
        print ("no results")
    
df_Canada_final.head (20)

/home/jupyterlab/conda/lib/python3.6/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


no results
no results


,Postcode,Borough,Neighbourhood,Address,Latitude,Longitude
0,M3A,North York,Parkwoods,"North York, M3A",43.753259,-79.329656
1,M4A,North York,Victoria Village,"North York, M4A",43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront,Regent Park","Downtown Toronto, M5A",0.000000,0.000000
3,M6A,North York,"Lawrence Heights,Lawrence Manor","North York, M6A",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,"Queen's Park, M7A",43.664366,-79.392328
5,M9A,Etobicoke,Islington Avenue,"Etobicoke, M9A",43.667856,-79.532242
6,M1B,Scarborough,"Rouge,Malvern","Scarborough, M1B",43.806686,-79.194353
7,M3B,North York,Don Mills North,"North York, M3B",43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens,Parkview Hill","East York, M4B",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson,Garden District","Downtown Toronto, M5B",43.657162,-79.378937


In [2]:
#df_Canada_final.to_csv('canada_final.csv')
dfCanadaCSV = pd.read_csv ("canada_final.csv")
dfCanada = dfCanadaCSV[dfCanadaCSV.Latitude!=0.0]
dfCanada.head ()

,Unnamed: 0,Postcode,Borough,Neighbourhood,Address,Latitude,Longitude
0,0,M3A,North York,Parkwoods,"North York, M3A",43.753259,-79.329656
1,1,M4A,North York,Victoria Village,"North York, M4A",43.725882,-79.315572
3,3,M6A,North York,"Lawrence Heights,Lawrence Manor","North York, M6A",43.718518,-79.464763
4,4,M7A,Queen's Park,Queen's Park,"Queen's Park, M7A",43.664366,-79.392328
5,5,M9A,Etobicoke,Islington Avenue,"Etobicoke, M9A",43.667856,-79.532242


In [3]:

address = 'Toronto, ON, Canada'
geolocator = Nominatim ()
location = geolocator.geocode (address)
latitude = location.latitude 
longitude = location.longitude
print ('The geographical coordinate of Toronto, Canada are: {},{} '.format(latitude,longitude))

/home/jupyterlab/conda/lib/python3.6/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


The geographical coordinate of Toronto, Canada are: 43.653963,-79.387207 


In [7]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)
# add markers to map
for lat, lng, postCode, borough, neighborhood in zip(dfCanada['Latitude'], 
                                                     dfCanada['Longitude'],
                                                     dfCanada['Postcode'],
                                                     dfCanada['Borough'], 
                                                     dfCanada['Neighbourhood']):
    label = '{} in {} {}'.format(neighborhood, borough,postCode)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        location=[lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

In [64]:
#Resuming each Borough according to number of Postcodes
resumedToronto =  dfCanada.groupby ('Borough').count()
resumedTorontoGroup = resumedToronto.drop('Unnamed: 0',1)
resumedTorontoGroup = resumedTorontoGroup.drop ('Neighbourhood',1)
resumedTorontoGroup = resumedTorontoGroup.drop ('Address',1)
resumedTorontoGroup = resumedTorontoGroup.drop ('Latitude',1)
resumedTorontoGroup = resumedTorontoGroup.drop ('Longitude',1)
resumedTorontoGroup

,Postcode
Borough,
Central Toronto,9
Downtown Toronto,17
East Toronto,5
East York,5
Etobicoke,12
Mississauga,1
North York,24
Queen's Park,1
Scarborough,17


In [65]:
# set number of clusters
kclusters = 6

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(resumedTorontoGroup)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([5, 1, 2, 2, 4, 0, 3, 0, 1, 2, 2], dtype=int32)

In [62]:
toronto_merged = resumedTorontoGroup
toronto_merged = toronto_merged.drop ('Postcode',1)
# add clustering labels
toronto_merged['Cluster Labels'] = kmeans.labels_
# merge resumedTorontoGroup with dfCanada to add latitude/longitude for each postcode
toronto_merged.reset_index()
toronto_merged = toronto_merged.join(dfCanada.set_index('Borough'), on='Borough')
toronto_merged = toronto_merged.drop ('Unnamed: 0',1)
toronto_merged.head(20) # check the last columns!

,Cluster Labels,Postcode,Neighbourhood,Address,Latitude,Longitude
Borough,,,,,,
Central Toronto,3,M4N,Lawrence Park,"Central Toronto, M4N",43.729196,-79.403292
Central Toronto,3,M5N,Roselawn,"Central Toronto, M5N",43.643466,-79.379099
Central Toronto,3,M4P,Davisville North,"Central Toronto, M4P",43.706980,-79.396276
Central Toronto,3,M5P,"Forest Hill North,Forest Hill West","Central Toronto, M5P",43.702580,-79.409696
Central Toronto,3,M4R,North Toronto West,"Central Toronto, M4R",43.706029,-79.402510
Central Toronto,3,M5R,"The Annex,North Midtown,Yorkville","Central Toronto, M5R",43.668191,-79.404947
Central Toronto,3,M4S,Davisville,"Central Toronto, M4S",43.696974,-79.395839
Central Toronto,3,M4T,"Moore Park,Summerhill East","Central Toronto, M4T",43.688186,-79.392944
Central Toronto,3,M4V,"Deer Park,Forest Hill SE,Rathnelly,South Hill,...","Central Toronto, M4V",43.687763,-79.394304


In [63]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        location=[lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters